In [6]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [7]:
OPENAI_API_KEY = '...'

In [ ]:
# pip install youtube-transcript-api
# pip install pytube

### 1. Simple Videos

In [2]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=True)

In [3]:
result = loader.load()

In [4]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>
Found video from Saturday Night Live that is 224 seconds long

[Document(page_content='LADIES AND GENTLEMEN, PEDRO PASCAL! [ CHEERS AND APPLAUSE ] >> THANK YOU, THANK YOU. THANK YOU VERY MUCH. I\'M SO EXCITED TO BE HERE. THANK YOU. I SPENT THE LAST YEAR SHOOTING A SHOW CALLED "THE LAST OF US" ON HBO. FOR SOME HBO SHOES, YOU GET TO SHOOT IN A FIVE STAR ITALIAN RESORT SURROUNDED BY BEAUTIFUL PEOPLE, BUT I SAID, NO, THAT\'S TOO EASY. I WANT TO SHOOT IN A FREEZING CANADIAN FOREST WHILE BEING CHASED AROUND BY A GUY WHOSE HEAD LOOKS LIKE A GENITAL WART. IT IS AN HONOR BEING A PART OF THESE HUGE FRANCHISEs LIKE "GAME OF THRONES" AND "STAR WARS," BUT I\'M STILL GETTING USED TO PEOPLE RECOGNIZING ME. THE OTHER DAY, A GUY STOPPED ME ON THE STREET AND SAYS, MY SON LOVES "THE MANDALORIAN" AND THE NEXT THING I KNOW, I\'M FACE TIMING WITH A 6-YEAR-OLD WHO HAS NO IDEA WHO I AM BECAUSE MY CHARACTER WEARS A MASK THE ENTIRE SHOW. THE GUY IS LIKE, DO THE MANDO VOICE, BUT IT\'S LIKE A BEDRO

In [8]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [9]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

' Pedro Pascal shared his experience shooting for HBO\'s "The Last of Us" and his journey to becoming a recognizable actor. He also spoke about his Chilean family, who were initially not proud of his acting career, but are now supportive. He concluded his speech by introducing Coldplay, who will be performing at the show.'

### 2. Long Videos

In [10]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)

In [11]:
result = loader.load()

In [12]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 14199 tokens (13943 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [27]:
texts = text_splitter.split_documents(result)

In [31]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer people who they don't they don't kno



> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

Covet shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves when the pandemic hit. They won contracts and became one of the core providers. They also created a snow machine to create a snow hill in their driveway for sledding without going to Tahoe.

 A company created in 2014 developed a vodka bottle with an LED sign that could be programmed with any message. When the COVID-19 pandemic hit, they shifted their production to safety equipment such as masks, respirators, gloves, and testing kits. They have since become a successful provider of safety equipment, winning contracts with the government and becoming the sole testing provider for the Lakers Stadium.


Levi's Stadium and other places are offering contracts worth over $100 million for products such as masks. During the COVID-19 pandemic, people were able to tak

'\n\nCovet shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves when the pandemic hit, winning contracts and becoming one of the core providers. They also created a snow machine to create a snow hill in their driveway for sledding. During the pandemic, people were able to take advantage of the opportunity to make a lot of money, with one friend making $2 million a month and another going public with 85 million in revenue. The speaker encouraged their father to take a risk and take action, which he learned from a Tony Robbins event.'

### 3. Multiple Videos

In [32]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [33]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' This article discusses the process of building a backyard golf green, including the cost of materials and the steps involved. Bella Ramsey and Pedro Pascal also discuss their experiences on the set of Game of Thrones and The Last of Us, including their first impressions of each other, what they took from set, and what item from the Game of Thrones world they would bring to The Last of Us. Bella talks about her most prized possession, a tear stick, and her best attribute in a survival scenario. Finally, they discuss their first jobs.'